# 네이버 영화평 감성 분석

In [1]:
# KoNLPy 설치 
!pip install konlpy

     |████████████████████████████████| 19.4MB 63.1MB/s 
     |████████████████████████████████| 460kB 48.9MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import files

uploaded = files.upload()

Saving ratings_test.txt to ratings_test.txt


In [6]:
uploaded.keys()

dict_keys(['ratings_test.txt'])

In [4]:
import numpy as np
import pandas as pd

In [8]:
test_df = pd.read_csv(list(uploaded.keys())[0], sep='\t')
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [10]:
uploaded = files.upload()

Saving ratings_train.txt to ratings_train.txt


In [11]:
train_df = pd.read_csv(list(uploaded.keys())[0], sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [13]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

### 데이터 전처리

In [12]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [14]:
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [15]:
# 0과 1의 분포
train_df.label.value_counts()

0    73342
1    72841
Name: label, dtype: int64

In [16]:
# Null값 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [17]:
# Null값 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

- 테스트 데이터 셋에도 적용

In [18]:
test_df.document.nunique()

49157

In [19]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [20]:
test_df.label.value_counts()

1    24712
0    24446
Name: label, dtype: int64

In [21]:
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [22]:
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

### 텍스트 전처리

In [23]:
# 한글 이외의 문자는 제거
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [24]:
# 영문이나 숫자로만 되어 있는 문장은 데이터가 없어짐
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [25]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- 테스트 데이터 셋

In [26]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'].replace('', np.nan, inplace=True)
test_df.isnull().sum()

id            0
document    162
label         0
dtype: int64

In [27]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

### 토큰화

In [29]:
from konlpy.tag import Okt
okt = Okt()

In [30]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [31]:
okt.morphs('교도소 이야기구먼 솔직히 재미는 없다평점 조정', stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [32]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_train.append(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_test.append(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [34]:
y_train = train_df.label.values
y_test = test_df.label.values

### Tfidf Vectorizer, Logistic Regression

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [36]:
tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tvect = tvect.transform(X_train)
X_test_tvect = tvect.transform(X_test)

In [37]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tvect, y_train)
pred = lr_clf.predict(X_test_tvect)
accuracy_score(y_test, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8305337279314216

### 실제 테스트

In [38]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ'
review2 = '이런 사랑영화가 다시 나올 수 있을까?'

In [40]:
import re
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review1)
review1

'진짜 개노잼이다 편이랑 같은 감독맞나러닝타임도 길어서 개지루함 ㄹㅇ'

In [41]:
morphs = okt.morphs(review1, stem=True)    # 토큰화
review = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
review

'진짜 개 노잼 이다 편이 랑 같다 감독 맞다 러닝 타임 기다 개 지루함 ㄹㅇ'

In [43]:
# review를 리스트로 만들어 주어야 함
review_tvect = tvect.transform([review])

In [44]:
pred = lr_clf.predict(review_tvect)

In [45]:
pred

array([0])

In [46]:
pred[0]

0

In [47]:
review2 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review2)
morphs = okt.morphs(review2, stem=True)    # 토큰화
review = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
review_tvect = tvect.transform([review])
pred = lr_clf.predict(review_tvect)
pred[0]

1

In [48]:
reviews = [review1, review2]
sentiments = ['부정', '긍정']

In [49]:
review_list = []
for review in reviews:
    review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review)
    morphs = okt.morphs(review, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    review_list.append(temp)

In [50]:
review_list_tvect = tvect.transform(review_list)
pred_list = lr_clf.predict(review_list_tvect)

In [51]:
pred_list[0], pred_list[1]

(0, 1)

In [53]:
sentiments[pred_list[0]], sentiments[pred_list[1]]

('부정', '긍정')

In [54]:
for i in range(2):
    print(f'{reviews[i]} - {sentiments[pred_list[i]]}')

진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ - 부정
이런 사랑영화가 다시 나올 수 있을까 - 긍정


### 임의의 데이터로 테스트

In [58]:
reviews = [
    '내 인생은 둘로 나뉜다. 죽은시인의 사회를 보기전과 그 후로....감동과 눈물..꼭보시길...',
    '우리나라의 강남을비롯 자식들을 혹독하게 가르치는 부모들에게 보여주고싶은 영화',
    '약간 지루하고 결말이 아쉬워요.',
    '영화를 보았으나 본 것 같지 않다.'
]

In [59]:
review_list = []
for review in reviews:
    review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review)
    morphs = okt.morphs(review, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    review_list.append(temp)

In [60]:
review_list_tvect = tvect.transform(review_list)
pred_list = lr_clf.predict(review_list_tvect)

In [61]:
for i in range(len(reviews)):
    print(f'{reviews[i]} - {sentiments[pred_list[i]]}')

내 인생은 둘로 나뉜다. 죽은시인의 사회를 보기전과 그 후로....감동과 눈물..꼭보시길... - 긍정
우리나라의 강남을비롯 자식들을 혹독하게 가르치는 부모들에게 보여주고싶은 영화 - 긍정
약간 지루하고 결말이 아쉬워요. - 부정
영화를 보았으나 본 것 같지 않다. - 긍정
